In [205]:
import gdown
import scipy.io
import numpy as np

file_id = "1XK_jSNNtkNCpR6-ygnQAhxsaBoe_gSxy"
url = f"https://drive.google.com/uc?id={file_id}"
output = "eeg_data.mat"
gdown.download(url, output, quiet=False)

#loading the train file
train = scipy.io.loadmat(output)

# print("MAT keys:", mat.keys())

file_id = "1XIudGXs_j_KLB_8V4b9lgKPXve0l9jJ5"
url = f"https://drive.google.com/uc?id={file_id}"
output_file = "eeg_data.mat"
gdown.download(url, output_file, quiet=False)

# loading the test file
test = scipy.io.loadmat(output_file)
# print("Keys in .mat file:", mat.keys())


Downloading...
From (original): https://drive.google.com/uc?id=1XK_jSNNtkNCpR6-ygnQAhxsaBoe_gSxy
From (redirected): https://drive.google.com/uc?id=1XK_jSNNtkNCpR6-ygnQAhxsaBoe_gSxy&confirm=t&uuid=9d0b5b81-b999-4a9e-bdab-1008d2c04739
To: /content/eeg_data.mat
100%|██████████| 14.5M/14.5M [00:00<00:00, 57.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1XIudGXs_j_KLB_8V4b9lgKPXve0l9jJ5
From (redirected): https://drive.google.com/uc?id=1XIudGXs_j_KLB_8V4b9lgKPXve0l9jJ5&confirm=t&uuid=0c60148f-c2bc-40e9-a9d5-57f1d1db5fc7
To: /content/eeg_data.mat
100%|██████████| 14.4M/14.4M [00:00<00:00, 43.1MB/s]


In [206]:
train

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Oct 12 13:45:36 2020',
 '__version__': '1.0',
 '__globals__': [],
 'fs': array([[256]], dtype=uint16),
 'trig': array([[ 0],
        [ 0],
        [ 0],
        ...,
        [-1],
        [-1],
        [-1]], dtype=int16),
 'y': array([[ 3.26827301e+02,  1.53989304e+02,  5.99201775e+01, ...,
          4.27079010e+02,  2.03007690e+02,  1.00998550e+02],
        [ 9.38830957e+03,  1.04189902e+04,  1.12646777e+03, ...,
         -1.83466272e+03,  1.02939171e+02, -1.69995837e+03],
        [-1.57571497e+03, -1.99006128e+03, -4.02767700e+02, ...,
          6.34901550e+02,  1.46847839e+02,  3.57204712e+02],
        ...,
        [-8.90913582e+00, -5.74011803e+01, -1.84253960e+01, ...,
         -8.97530594e+01, -4.57585716e+01, -6.61705399e+01],
        [-6.31855507e+01, -6.80609741e+01, -1.92778454e+01, ...,
         -8.74048996e+01, -4.37283249e+01, -9.39249649e+01],
        [-3.03183117e+01, -2.88373718e+01,  1.90799122e+

In [207]:
y_train = train['y']
trig_train=train['trig']

y_test = test['y']
trig_test=test['trig']


In [208]:
np.shape(y_train)

(205536, 16)

In [209]:
np.shape(y_test)

(199552, 16)

In [210]:
import numpy as np

def segment_trials(train_y: np.ndarray, train_trig: np.ndarray):
    flat_trig = train_trig[:, 0]

    segments = []
    labels = []

    i = 0
    N = flat_trig.shape[0]
    while i < N:
        if flat_trig[i] == 0:
            i += 1
            continue

        current_label = int(flat_trig[i])
        start = i
        # consume until it changes
        while i < N and flat_trig[i] == current_label:
            i += 1
        end = i
        # slice out the corresponding rows of train_y
        segments.append(train_y[start:end, :])
        labels.append(current_label)

    return segments, labels

# segments, labels = segment_trials(y_train, trig_train)
# for seg, lbl in zip(segments, labels):
#     print(f"Segment of {seg.shape[0]} samples, label = {lbl}")


In [211]:
y_train, trig_train = segment_trials(y_train, trig_train)
y_test, trig_test = segment_trials(y_test, trig_test)

In [212]:
np.shape(y_train)

(80, 2048, 16)

In [213]:
y_train=np.array(y_train)
y_test=np.array(y_test)
trig_train=np.array(trig_train)
trig_test=np.array(trig_test)

In [214]:
len(trig_test)

80

In [215]:
np.shape(y_train)

(80, 2048, 16)

In [216]:
from scipy.signal import butter, filtfilt
y_train = y_train.transpose(0, 2, 1)  # (epochs, channels, samples)
y_test = y_test.transpose(0, 2, 1)  # (epochs, channels, samples)
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data, axis=-1)

# Applying BPF: 8-30 Hz
fs = 256  # Hz
lowcut = 8
highcut = 30
y_train= bandpass_filter(y_train, lowcut, highcut, fs)
y_test= bandpass_filter(y_test, lowcut, highcut, fs)

In [217]:
from sklearn.preprocessing import StandardScaler

# Flatten for normalization if needed
y_train_flat = y_train.reshape((y_train.shape[0], -1))
y_test_flat = y_test.reshape((y_test.shape[0], -1))

# Fit scaler on training data and transform both train and test
scaler = StandardScaler()
y_train_normalized = scaler.fit_transform(y_train_flat)
y_test_normalized = scaler.transform(y_test_flat)

# Reshape back to original shape
y_train = y_train_normalized.reshape(y_train.shape)
y_test = y_test_normalized.reshape(y_test.shape)


In [218]:
def compute_spd_matrices(eeg_data):

    epochs, channels, samples = eeg_data.shape
    spd_matrices = np.zeros((epochs, channels, channels))

    for i in range(epochs):
        epoch = eeg_data[i]  # shape (channels, samples)
        cov_matrix = np.cov(epoch)  # shape (channels, channels)
        # Optional: ensure SPD by adding a small regularization term to the diagonal
        cov_matrix += 1e-6 * np.eye(channels)
        spd_matrices[i] = cov_matrix

    return spd_matrices

# Example usage:
y_train_spd = compute_spd_matrices(y_train)
y_test_spd = compute_spd_matrices(y_test)


In [219]:
np.shape(y_train_spd)

(80, 16, 16)

In [222]:
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

# Tangent space mapping
ts = TangentSpace(metric='riemann')

# Fit TangentSpace mapper to training SPD matrices and transform both sets
X_train_tangent = ts.fit_transform(y_train_spd)
X_test_tangent = ts.transform(y_test_spd)

# Use LDA for classification
lda = LDA()
lda.fit(X_train_tangent, trig_train)

# Predict and evaluate
y_pred = lda.predict(X_test_tangent)
acc = accuracy_score(trig_test, y_pred)

print(f"Tangent space + LDA accuracy: {acc:.4f}")
print(classification_report(trig_test, y_pred))


Tangent space + LDA accuracy: 0.5750
              precision    recall  f1-score   support

          -1       0.80      0.20      0.32        40
           1       0.54      0.95      0.69        40

    accuracy                           0.57        80
   macro avg       0.67      0.57      0.51        80
weighted avg       0.67      0.57      0.51        80



In [223]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV

# Define LDA and param grid
lda = LinearDiscriminantAnalysis()
param_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto'],
}

# Grid Search
lda_search = GridSearchCV(lda, param_grid, cv=5, scoring='accuracy')
lda_search.fit(X_train_tangent, trig_train)

# Results
print("Best LDA parameters:", lda_search.best_params_)
print(f"Best Cross-Val Accuracy: {lda_search.best_score_:.4f}")

# Evaluate on test set
best_lda = lda_search.best_estimator_
y_pred = best_lda.predict(X_test_tangent)

from sklearn.metrics import accuracy_score, classification_report
print(f"Test Accuracy: {accuracy_score(trig_test, y_pred):.4f}")
print(classification_report(trig_test, y_pred))


Best LDA parameters: {'shrinkage': 'auto', 'solver': 'lsqr'}
Best Cross-Val Accuracy: 0.9000
Test Accuracy: 0.5000
              precision    recall  f1-score   support

          -1       0.50      1.00      0.67        40
           1       0.00      0.00      0.00        40

    accuracy                           0.50        80
   macro avg       0.25      0.50      0.33        80
weighted avg       0.25      0.50      0.33        80



/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py", line 716, in fit
    self._solve_eigen(
  File

In [224]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

grid_search.fit(X_train_tangent, trig_train)

print("Best Parameters:", grid_search.best_params_)
print(f"Best Cross-Val Accuracy: {grid_search.best_score_:.4f}")

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_tangent)

print(f"Test Accuracy: {accuracy_score(trig_test, y_pred):.4f}")
print(classification_report(trig_test, y_pred))


In [225]:
np.shape(trig_test)

(80,)

In [227]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
trig_train = le.fit_transform(trig_train)
trig_test = le.transform(trig_test)


In [228]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import numpy as np

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert your NumPy data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_tangent, dtype=torch.float32)
# y_train_tensor = torch.tensor(trig_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_tangent, dtype=torch.float32)
# y_test_tensor = torch.tensor(trig_test, dtype=torch.long)

y_train_tensor = torch.tensor(trig_train, dtype=torch.long)
y_test_tensor = torch.tensor(trig_test, dtype=torch.long)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model definition
class SPDClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate, output_dim):
        super(SPDClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Params to search
input_dim = X_train_tensor.shape[1]
output_dim = len(np.unique(trig_train))

hidden_dims = [64, 128]
dropouts = [0.2, 0.3]
learning_rates = [1e-2, 1e-3]
batch_sizes = [32, 64]
epochs = 20  # Use higher if needed

best_test_acc = 0.0
best_config = {}

# Begin grid search
for hd in hidden_dims:
    for dr in dropouts:
        for lr in learning_rates:
            for bs in batch_sizes:
                print(f"\n🔍 Trying config: hidden={hd}, dropout={dr}, lr={lr}, batch={bs}")

                # Create model
                model = SPDClassifier(input_dim, hd, dr, output_dim).to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                criterion = nn.CrossEntropyLoss()
                train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=bs, shuffle=True)

                for epoch in range(epochs):
                    model.train()
                    for xb, yb in train_loader:
                        xb, yb = xb.to(device), yb.to(device)
                        optimizer.zero_grad()
                        preds = model(xb)
                        loss = criterion(preds, yb)
                        loss.backward()
                        optimizer.step()

                    # Evaluate both train and test accuracy
                    model.eval()
                    with torch.no_grad():
                        train_outputs = model(X_train_tensor.to(device))
                        train_preds = torch.argmax(train_outputs, dim=1)
                        train_acc = accuracy_score(y_train_tensor.numpy(), train_preds.cpu().numpy())

                        test_outputs = model(X_test_tensor.to(device))
                        test_preds = torch.argmax(test_outputs, dim=1)
                        test_acc = accuracy_score(y_test_tensor.numpy(), test_preds.cpu().numpy())

                    print(f"Epoch {epoch+1}/{epochs} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

                    # Save best config
                    if test_acc > best_test_acc:
                        best_test_acc = test_acc
                        best_config = {
                            'hidden_dim': hd,
                            'dropout': dr,
                            'learning_rate': lr,
                            'batch_size': bs,
                            'epoch': epoch + 1,
                            'train_acc': train_acc,
                            'test_acc': test_acc
                        }

print("\n Best configuration:")
print(best_config)


🔍 Trying config: hidden=64, dropout=0.2, lr=0.01, batch=32
Epoch 1/20 | Train Acc: 0.9000 | Test Acc: 0.4875
Epoch 2/20 | Train Acc: 0.9000 | Test Acc: 0.4875
Epoch 3/20 | Train Acc: 0.9250 | Test Acc: 0.4875
Epoch 4/20 | Train Acc: 0.9500 | Test Acc: 0.5125
Epoch 5/20 | Train Acc: 0.9625 | Test Acc: 0.5250
Epoch 6/20 | Train Acc: 0.9875 | Test Acc: 0.5375
Epoch 7/20 | Train Acc: 1.0000 | Test Acc: 0.5375
Epoch 8/20 | Train Acc: 1.0000 | Test Acc: 0.5750
Epoch 9/20 | Train Acc: 1.0000 | Test Acc: 0.5750
Epoch 10/20 | Train Acc: 1.0000 | Test Acc: 0.5750
Epoch 11/20 | Train Acc: 1.0000 | Test Acc: 0.5750
Epoch 12/20 | Train Acc: 1.0000 | Test Acc: 0.6125
Epoch 13/20 | Train Acc: 1.0000 | Test Acc: 0.6125
Epoch 14/20 | Train Acc: 1.0000 | Test Acc: 0.5875
Epoch 15/20 | Train Acc: 1.0000 | Test Acc: 0.5875
Epoch 16/20 | Train Acc: 1.0000 | Test Acc: 0.5875
Epoch 17/20 | Train Acc: 1.0000 | Test Acc: 0.6000
Epoch 18/20 | Train Acc: 1.0000 | Test Acc: 0.6000
Epoch 19/20 | Train Acc: 1.0000